In [7]:
#Importing the libraries
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score , mean_squared_error

In [8]:
data = pd.read_csv("LA_Crime_dataset.csv")

In [9]:
data.isna().sum()

DR_NO                   0
Date Rptd               0
DATE OCC                0
TIME OCC                0
AREA                    0
AREA NAME               0
Rpt Dist No             0
Part 1-2                0
Crm Cd                  0
Crm Cd Desc             0
Mocodes            151619
Vict Age                0
Vict Sex           144644
Vict Descent       144656
Premis Cd              16
Premis Desc           588
Weapon Used Cd     677744
Weapon Desc        677744
Status                  1
Status Desc             0
Crm Cd 1               11
Crm Cd 2           935831
Crm Cd 3          1002677
Crm Cd 4          1004927
LOCATION                0
Cross Street       850755
LAT                     0
LON                     0
dtype: int64

In [5]:
data[["AREA","AREA NAME"]].head()

,AREA,AREA NAME
0,15,N Hollywood
1,15,N Hollywood
2,9,Van Nuys
3,7,Wilshire
4,14,Pacific


In [10]:
data["Premis Desc"]

0          SINGLE FAMILY DWELLING
1                        SIDEWALK
2          SINGLE FAMILY DWELLING
3                          STREET
4                           ALLEY
                    ...          
1004986                    STREET
1004987    SINGLE FAMILY DWELLING
1004988                    STREET
1004989               HIGH SCHOOL
1004990               HIGH SCHOOL
Name: Premis Desc, Length: 1004991, dtype: object